In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score

In [28]:
df = pd.read_csv(r'C:\Users\sanji\Downloads\TotalFBREFPlayerData - Sheet1.csv')
df

,player_name,age,position,club_name,club_involved_name,transfer_movement,matches_played,goals,assists,yellow_card,...,carries,goals_against,goals_against90,Saves,Saves%,PSxG,plus_minus90,season,nationality,fee_cleaned
0,Jose Cholevas,31,Left-Back,Watford FC,AS Roma,in,37,2.0,0.0,6.0,...,1588.0,0.0,0.0,0.0,0.0,0.0,0.75,2015/2016,England,2.50
1,JosÃ© Crespo,28,Centre-Back,Aston Villa,CÃ³rdoba CF,in,33,1.0,4.0,4.0,...,1526.0,0.0,0.0,0.0,0.0,0.0,0.99,2015/2016,England,0.80
2,Luis HernÃ¡ndez,27,Centre-Back,Leicester City,MÃ¡laga CF,out,25,2.0,3.0,0.0,...,182.0,0.0,0.0,0.0,0.0,0.0,0.17,2016/2017,England,2.00
3,Roberto Pereyra,25,Attacking Midfield,Watford FC,Juventus,in,39,0.0,1.0,2.0,...,1534.0,0.0,0.0,0.0,0.0,0.0,0.48,2016/2017,England,13.50
4,Antonio BarragÃ¡n,29,Right-Back,Middlesbrough FC,Valencia,in,34,5.0,1.0,5.0,...,85.0,0.0,0.0,0.0,0.0,0.0,0.12,2016/2017,England,2.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,Matt Targett,26,Left-Back,Aston Villa,Newcastle United,in,38,0.0,2.0,7.0,...,1151.0,0.0,0.0,0.0,0.0,0.0,0.29,2022/2023,England,17.50
417,Craig Dawson,31,Centre-Back,Watford FC,West Ham United,in,24,4.0,0.0,5.0,...,522.0,0.0,0.0,0.0,0.0,0.0,0.59,2021/2022,England,2.30
418,Craig Dawson,32,Centre-Back,West Ham United,Wolverhampton Wanderers,in,50,4.0,2.0,7.0,...,1281.0,0.0,0.0,0.0,0.0,0.0,0.36,2022/2023,England,3.75
419,Danny Ings,29,Centre-Forward,Southampton FC,Aston Villa,in,33,13.0,4.0,1.0,...,459.0,0.0,0.0,0.0,0.0,0.0,-0.29,2021/2022,England,29.42


In [29]:
df = df.drop(['player_name'], axis=1)
df['minutes'] = df['minutes'].str.replace(',', '').astype(float)
df = df.fillna(df.mean(numeric_only=True))

In [30]:
new_df = df.copy()  

out_rows = df[df['transfer_movement'] == 'out'].index

df_modified = df.copy()

for i in out_rows:
    df_modified.loc[i, ['club_involved_name', 'club_name']] = \
        df.loc[i, ['club_name', 'club_involved_name']].values

df_modified.loc[out_rows, 'transfer_movement'] = "in"

df = df_modified.copy()
df = df.drop('transfer_movement', axis=1)

In [31]:
Q1 = df["fee_cleaned"].quantile(0.25)
Q3 = df["fee_cleaned"].quantile(0.75)
IQR = Q3 - Q1
df = df[~((df["fee_cleaned"] < (Q1 - 1.5 * IQR)) | (df["fee_cleaned"] > (Q3 + 1.5 * IQR)))]

df.dropna(subset=["fee_cleaned"], inplace=True)

In [32]:
df

,age,position,club_name,club_involved_name,matches_played,goals,assists,yellow_card,red_card,90s_played,...,carries,goals_against,goals_against90,Saves,Saves%,PSxG,plus_minus90,season,nationality,fee_cleaned
0,31,Left-Back,Watford FC,AS Roma,37,2.0,0.0,6.0,1.0,33.0,...,1588.0,0.0,0.0,0.0,0.0,0.0,0.75,2015/2016,England,2.50
1,28,Centre-Back,Aston Villa,CÃ³rdoba CF,33,1.0,4.0,4.0,0.0,30.0,...,1526.0,0.0,0.0,0.0,0.0,0.0,0.99,2015/2016,England,0.80
2,27,Centre-Back,MÃ¡laga CF,Leicester City,25,2.0,3.0,0.0,0.0,11.0,...,182.0,0.0,0.0,0.0,0.0,0.0,0.17,2016/2017,England,2.00
3,25,Attacking Midfield,Watford FC,Juventus,39,0.0,1.0,2.0,0.0,37.0,...,1534.0,0.0,0.0,0.0,0.0,0.0,0.48,2016/2017,England,13.50
4,29,Right-Back,Middlesbrough FC,Valencia,34,5.0,1.0,5.0,0.0,26.0,...,85.0,0.0,0.0,0.0,0.0,0.0,0.12,2016/2017,England,2.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,26,Left-Back,Aston Villa,Newcastle United,38,0.0,2.0,7.0,0.0,38.0,...,1151.0,0.0,0.0,0.0,0.0,0.0,0.29,2022/2023,England,17.50
417,31,Centre-Back,Watford FC,West Ham United,24,4.0,0.0,5.0,1.0,23.7,...,522.0,0.0,0.0,0.0,0.0,0.0,0.59,2021/2022,England,2.30
418,32,Centre-Back,West Ham United,Wolverhampton Wanderers,50,4.0,2.0,7.0,1.0,47.1,...,1281.0,0.0,0.0,0.0,0.0,0.0,0.36,2022/2023,England,3.75
419,29,Centre-Forward,Southampton FC,Aston Villa,33,13.0,4.0,1.0,0.0,28.0,...,459.0,0.0,0.0,0.0,0.0,0.0,-0.29,2021/2022,England,29.42


In [33]:
position_type = ['Attacker' if position in ['Centre-Forward', 'Second Striker', 'Left Winger',
                                            'Right Winger'] else 'Midfielder' if position in ['Attacking Midfield',
                                                                                              'Central Midfield',
                                                                                              'Right Midfield',
                                                                                              'Defensive Midfield'] else 'Defender' if position in [
    'Centre-Back', 'Right-Back', 'Left-Back'] else 'Goalkeeper' if position == 'Goalkeeper' else None for position in
                 df['position']]

position_idx = df.columns.get_loc('position')

df.insert(position_idx + 1, 'position_type', position_type)
df

,age,position,position_type,club_name,club_involved_name,matches_played,goals,assists,yellow_card,red_card,...,carries,goals_against,goals_against90,Saves,Saves%,PSxG,plus_minus90,season,nationality,fee_cleaned
0,31,Left-Back,Defender,Watford FC,AS Roma,37,2.0,0.0,6.0,1.0,...,1588.0,0.0,0.0,0.0,0.0,0.0,0.75,2015/2016,England,2.50
1,28,Centre-Back,Defender,Aston Villa,CÃ³rdoba CF,33,1.0,4.0,4.0,0.0,...,1526.0,0.0,0.0,0.0,0.0,0.0,0.99,2015/2016,England,0.80
2,27,Centre-Back,Defender,MÃ¡laga CF,Leicester City,25,2.0,3.0,0.0,0.0,...,182.0,0.0,0.0,0.0,0.0,0.0,0.17,2016/2017,England,2.00
3,25,Attacking Midfield,Midfielder,Watford FC,Juventus,39,0.0,1.0,2.0,0.0,...,1534.0,0.0,0.0,0.0,0.0,0.0,0.48,2016/2017,England,13.50
4,29,Right-Back,Defender,Middlesbrough FC,Valencia,34,5.0,1.0,5.0,0.0,...,85.0,0.0,0.0,0.0,0.0,0.0,0.12,2016/2017,England,2.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,26,Left-Back,Defender,Aston Villa,Newcastle United,38,0.0,2.0,7.0,0.0,...,1151.0,0.0,0.0,0.0,0.0,0.0,0.29,2022/2023,England,17.50
417,31,Centre-Back,Defender,Watford FC,West Ham United,24,4.0,0.0,5.0,1.0,...,522.0,0.0,0.0,0.0,0.0,0.0,0.59,2021/2022,England,2.30
418,32,Centre-Back,Defender,West Ham United,Wolverhampton Wanderers,50,4.0,2.0,7.0,1.0,...,1281.0,0.0,0.0,0.0,0.0,0.0,0.36,2022/2023,England,3.75
419,29,Centre-Forward,Attacker,Southampton FC,Aston Villa,33,13.0,4.0,1.0,0.0,...,459.0,0.0,0.0,0.0,0.0,0.0,-0.29,2021/2022,England,29.42


In [34]:
copy = df.copy()

le = LabelEncoder()
copy['position'] = le.fit_transform(copy['position'])
copy['position_type'] = le.fit_transform(copy['position_type'])
copy['season'] = le.fit_transform(copy['season'])
copy['nationality'] = le.fit_transform(copy['nationality'])

unique_club_names = pd.concat([copy['club_involved_name'], copy['club_name']]).unique()

le = LabelEncoder()

club_series = le.fit_transform(unique_club_names)

club_mapping = {club_name: i for club_name, i in zip(unique_club_names, club_series)}

copy['club_involved_name'] = copy['club_involved_name'].map(club_mapping)
copy['club_name'] = copy['club_name'].map(club_mapping)
df = copy.copy()
df

,age,position,position_type,club_name,club_involved_name,matches_played,goals,assists,yellow_card,red_card,...,carries,goals_against,goals_against90,Saves,Saves%,PSxG,plus_minus90,season,nationality,fee_cleaned
0,31,8,1,180,7,37,2.0,0.0,6.0,1.0,...,1588.0,0.0,0.0,0.0,0.0,0.0,0.75,0,23,2.50
1,28,2,1,16,50,33,1.0,4.0,4.0,0.0,...,1526.0,0.0,0.0,0.0,0.0,0.0,0.99,0,23,0.80
2,27,2,1,108,94,25,2.0,3.0,0.0,0.0,...,182.0,0.0,0.0,0.0,0.0,0.0,0.17,1,23,2.00
3,25,0,3,180,86,39,0.0,1.0,2.0,0.0,...,1534.0,0.0,0.0,0.0,0.0,0.0,0.48,1,23,13.50
4,29,12,1,104,172,34,5.0,1.0,5.0,0.0,...,85.0,0.0,0.0,0.0,0.0,0.0,0.12,1,23,2.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,26,8,1,16,110,38,0.0,2.0,7.0,0.0,...,1151.0,0.0,0.0,0.0,0.0,0.0,0.29,7,23,17.50
417,31,2,1,180,183,24,4.0,0.0,5.0,1.0,...,522.0,0.0,0.0,0.0,0.0,0.0,0.59,6,23,2.30
418,32,2,1,183,185,50,4.0,2.0,7.0,1.0,...,1281.0,0.0,0.0,0.0,0.0,0.0,0.36,7,23,3.75
419,29,3,0,152,16,33,13.0,4.0,1.0,0.0,...,459.0,0.0,0.0,0.0,0.0,0.0,-0.29,6,23,29.42


In [35]:
X = df.drop(['fee_cleaned'], axis=1)
y = df['fee_cleaned']

In [36]:
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

mse_scores = []
r2_scores = []

In [37]:
gradBoost = GradientBoostingRegressor(max_depth=3)

for train_index, test_index in kf.split(X):
   
    features_train, features_test = X.iloc[train_index], X.iloc[test_index]
    target_train, target_test = y.iloc[train_index], y.iloc[test_index]

    gradBoost.fit(features_train, target_train)
    target_pred = gradBoost.predict(features_test)

    mse = mean_squared_error(target_test, target_pred)
    r2 = r2_score(target_test, target_pred)

    mse_scores.append(mse)
    r2_scores.append(r2)

average_mse = np.mean(mse_scores)
average_r2 = np.mean(r2_scores)

print("Average Mean Squared Error:", average_mse)
print("Average R-squared:", average_r2)

Average Mean Squared Error: 120.42384093869683
Average R-squared: 0.1011858578328807


In [38]:
import joblib

joblib.dump(gradBoost, 'GradientBoostingRegressor_model.joblib')

['GradientBoostingRegressor_model.joblib']